# 映画のレビュー検索
- 有用なレビューデータの抽出

In [ ]:
import MeCab
import csv
import re
import json

remove=[]
result={}

with open('/mnt/data/movie_ds/data/rankings.CSV') as f:

    #辞書形式
    f_matome = csv.reader(f, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)

    for i,text in enumerate(f_matome):
        V_count=0
        mecab = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
        par = mecab.parseToNode(text[2])
        if re.compile('\d{4}').search(text[2]):
            remove.append(text[2])
        elif re.compile('ホラー|コメディ|ドラマ|サスペンス|アクション|SF|ＳＦ|ミュージカル').search(text[2]):
            result[text[0]]=text[2]
        elif re.compile('(([mMｍＭ][yYｙＹ])|マイ|まい|ﾏｲ)(・|\s)*(best|Best|BEST|ｂｅｓｔ|Ｂｅｓｔ|ＢＥＳＴ|worst|Worst|WORST|ＷＯＲＳＴ|Ｗｏｒｓｔ|ｗｏｒｓｔ|([べベ][すス][とト])|ﾍﾞｽﾄ|わーすと|ワースト|ﾜｰｽﾄ)').search(text[2]):
            while par:
                if par.feature.split(',')[0] == '動詞':
                    V_count+=1
                par = par.next
            if V_count!=0:
                result[text[0]]=text[2]
            else:
                remove.append(text[2])
        else:
            result[text[0]]=text[2]
with open('matome_selected.json', mode='w', encoding='utf-8') as file:
    json.dump(result, file, ensure_ascii=False, indent=2)

- 映画のまとめデータの整理

In [ ]:
import pprint

with open('/mnt/data/movie_ds/data/rankings_cinemas.CSV') as f:
    f_movie = csv.reader(f, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
    matome={}
    last_id=0
    
    for i,now in enumerate(f_movie):
       
        if now[1] in matome.keys():
            matome[now[1]].append(now[3])
        else:
            matome[now[1]]=[now[3]]
        
with open('matome_dic.json', mode='w', encoding='utf-8') as file:
    json.dump(matome, file, ensure_ascii=False, indent=2)

- movie idと映画名の対応付け

In [ ]:
with open('/mnt/data/movie_ds/data/cinemas.CSV') as f:
    movie = csv.reader(f, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
    movie_dic={}
    
    for i,now in enumerate(movie):
        movie_dic[now[0]]=now[1]
with open('movie_dic.json', mode='w', encoding='utf-8') as file:
    json.dump(movie_dic, file, ensure_ascii=False, indent=2)

- 各映画の含まれてるまとめを映画ごとにまとめる

In [ ]:
from pprint import pprint
movie_rank={}
for k,v in matome.items():
    for m in v:
        if m in movie_rank.keys():
            movie_rank[m].append(k)
        else:
            movie_rank[m]=[k]
sortedDict = sorted(movie_rank.items(), key=lambda x: len(x[1]))
sort_dic=[]
for k,v in sortedDict:
    if k in movie_dic.keys():
        matome_title=[]
        for matome_id in v:
            if matome_id in result.keys():
                matome_title.append(result[matome_id].replace('　',' '))
        sort_dic.append((movie_dic[k].replace('　',' '),matome_title))
with open('sorted_dic.json', mode='w', encoding='utf-8') as file:
    json.dump(sort_dic, file, ensure_ascii=False, indent=2)

In [ ]:
from pprint import pprint
import csv
import re
import json

json_open = open('matome_dic.json', 'r')
matome = json.load(json_open)
movie_rank={}
for k,v in matome.items():
    for m in v:
        if m in movie_rank.keys():
            movie_rank[m].append(k)
        else:
            movie_rank[m]=[k]
sortedDict = sorted(movie_rank.items(), key=lambda x: len(x[1]))
with open('movie_to_matome.json', mode='w', encoding='utf-8') as file:
    json.dump(sortedDict, file, ensure_ascii=False, indent=2)

- 映画のレビューデータの整理

In [ ]:
from pprint import pprint
import csv
import json

with open('/mnt/data/movie_ds/data/reviews.CSV') as f:
    f_review = csv.reader(f, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
    review={}
    last_id=0
    
    for i,now in enumerate(f_review):
       
        if now[1] in review.keys():
            review[now[1]].append(now[7])
        else:
            review[now[1]]=[now[7]]
    for k,v in review.items():
        review[k]=[''.join(v)]
        
with open('review_dic.json', mode='w', encoding='utf-8') as file:
    json.dump(review, file, ensure_ascii=False, indent=2)

- BERTを使用する前準備

In [ ]:
import re
import os
import random
import collections
import time
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import transformers
import mojimoji
from pathlib import Path
from tqdm import tqdm
from torch.utils.data import DataLoader, \
  RandomSampler, SequentialSampler, TensorDataset

def text2bertinput(
        input_text=None, 
        tokenizer=None,
        max_seq_len=None):
    #入力テキスト内の半角を全角に変換する
    input_text = mojimoji.han_to_zen(input_text)
    #特殊トークンの付与＆サブワード分割
    tokenize_txt = tokenizer.tokenize('[CLS]' + input_text + '[SEP]')
    #各id列の作成
    token_ids = tokenizer.convert_tokens_to_ids(tokenize_txt)
    mask_ids = [1] * len(token_ids)
    segment_ids = [0] * len(token_ids)
    #入力テキストの長さがモデルが指定する最大入力長になるまで[PAD]で水増し
    while len(token_ids) < max_seq_len:
        #[PAD]トークンの付与
        token_ids.append(0)
        mask_ids.append(0)
        segment_ids.append(0)
    #入力テキストが既に最大入力長を超えている場合、途中で切る。
    token_ids = token_ids[:max_seq_len]
    mask_ids = mask_ids[:max_seq_len]
    segment_ids = segment_ids[:max_seq_len]
    #各idの長さが適切かどうかの確認
    assert len(token_ids) == max_seq_len
    assert len(mask_ids) == max_seq_len
    assert len(segment_ids) == max_seq_len
    #各id列をtorch.tensorへ
    tensor_token_id = torch.tensor([token_ids], dtype=torch.long)
    tensor_mask_id = torch.tensor([mask_ids], dtype=torch.long)
    tensor_segment_id = torch.tensor([segment_ids], dtype=torch.long)
    return  {'token_ids': tensor_token_id,
             'mask_ids': tensor_mask_id,
             'segment_ids': tensor_segment_id}

#使用するクラス
class GetBertOutputVec(transformers.BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = transformers.BertModel(config)
        self.init_weights()
        #Average_pooling層の定義
        self.adaptive_avg_layer = nn.AdaptiveAvgPool2d((1, 768))
         
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        output_attentions=None,
        output_hidden_states=None,
        labels=None,
        output_vec='cls',
        return_dict=None
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        '''
        引数で指定された各id列をBERTに入力する処理------------------------------
        '''
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        '''
        BERTモデルから出力されたベクトルに対する処理----------------------------
        '''
        sequence_output = None
        #3-1-a_CLSトークンに対応したベクトルのみを取得したい場合
        if output_vec == 'cls':
            #outputs[1]は[CLS]トークンに対応したベクトル
            #shapeは[batch_size, 768]
            sequence_output = outputs[1]
              
        #3-1-b_平均ベクトルを取得したい場合
        if output_vec == 'avg_pooling':
            #outputs[0]は全ての入力トークンに対応したベクトルが格納されている
            #shapeは[batch_size, 入力長, 768]
            pooled_output = outputs[0]
            #入力トークンの内、特殊トークン以外のトークンに対応したベクトルの平均を求める
            #sequence_outputのshapeは[batch_size, 768]
            sequence_output = self.get_average_pooling(input_ids, pooled_output)  
        '''
        sequence_output([batch_size, 768])を返す--------------------------------
        '''
        return sequence_output

    #平均ベクトルを計算するための関数
    def get_average_pooling(self, input_ids, pooled_output):
        average_vectors = []
        nums_ctos_token = []
        batch_size = 0
        #3-1-2_ミニバッチからデータを取り出し、CLSからSEPまでのトークンの数を数える
        for input_id in input_ids:
            input_id = input_id[torch.where(input_id != 0)]
            nums_ctos_token.append(len(input_id))
        #3-1-3_ミニバッチからデータを一つずつ取り出し、それぞれの平均ベクトルを算出
        for output_matrix, num_ctos_token in zip(pooled_output, nums_ctos_token):
            #3-1-4_特殊トークン([CLS], [SEP], [PAD])に対応したベクトルの削除
            purpose_matrix = output_matrix[1:num_ctos_token-1]
            purpose_matrix = purpose_matrix.view(-1, num_ctos_token-2, 768)
            #3-1-5_PADが除かれたマトリクスを平均プーリング層へ入力
            average_vector = self.adaptive_avg_layer(purpose_matrix)
            average_vectors.append(average_vector)
            batch_size += 1
        #3-1-6_各ベクトルを持つリストをtensorに変換
        average_vectors = torch.cat(average_vectors, dim=1)
        average_vectors = average_vectors.view(-1)
        #3-1-7_ミニバッチに整える。
        average_vectors = average_vectors.view(batch_size, 768)
        return average_vectors

#Tokenizerの定義
from transformers import BertJapaneseTokenizer

pretrained_path = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_path)

#GPUの定義
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

# modelの定義
model = GetBertOutputVec.from_pretrained(pretrained_path)

#モデルをgpuへ
model.to(device)


- レビューをBERTでベクトル化

In [ ]:
import GPUtil
import pickle
import json
from pymongo import MongoClient
from bson.binary import Binary
import re

#関数を使ってテキストの前処理を行う
client=MongoClient('127.0.0.1',27017) #ip, port
db=client["review_data"] #database name
collection=db["review_vector"] #collection name
json_open = open('review_dic.json', 'r')
review = json.load(json_open)
start_count=1
csv_header = ['movie_id','review_id','review_num','review_body','vector']
for k,v in review.items():
    review_count=1
    for now in v:
        section_count=1
        target_text = re.sub('[「」（）()『』]','',now)
        target_text=target_text.replace('\\n','').replace(' ','')
        target_seq=re.split('[。?？!！☆♡]',target_text)
        for now_seq in target_seq:
            review_vector={}
            if len(now_seq)!=0:
                with torch.no_grad():
                    with open('log.txt', 'w') as fi:
                        #確認
                #         print('token_ids', target_ids['token_ids'])
                #         print('mask_ids', target_ids['mask_ids'])
                #         print('segment_ids', target_ids['segment_ids'])

                        target_ids = text2bertinput(input_text=now_seq,tokenizer=tokenizer,max_seq_len=512)

                        #各入力idをgpuへ
                        token_ids = target_ids['token_ids'].to(device)
                        mask_ids = target_ids['mask_ids'].to(device)
                        segment_ids = target_ids['segment_ids'].to(device)

                        #modelに各idを入力する
                        try:
                            bert_ouputs_average = model(
                                    input_ids = token_ids,
                                    attention_mask = mask_ids,
                                    token_type_ids = segment_ids,
                                    output_vec = 'avg_pooling')
                            bert_ouput_vecs_non_grad = bert_ouputs_average.detach()
                            review_vector['id']=start_count
                            review_vector['movie_id']=k
                            review_vector['review_id']=review_count
                            review_vector['review_num']=section_count
                            review_vector['review_body']=now_seq
                            review_vector['vector']=Binary(pickle.dumps(bert_ouput_vecs_non_grad[0], protocol=2))
                            collection.insert_one(review_vector)
                            del token_ids
                            del mask_ids
                            del segment_ids
                            torch.cuda.empty_cache()
                        except Exception as e:
                            print(e)
                            print(type(e))
                    start_count+=1
            section_count+=1
        review_count+=1
#     if start_count > 10:
#         break

- 各レビューのベクトルを映画ごとの平均ベクトルに変換

In [ ]:
client=MongoClient('127.0.0.1',27017) #ip, port
db=client["review_data"] #database name
collection=db["review_vector"] #collection name
collection2=db["movie_vector"] #collection name
json_open = open('movie_dic.json', 'r')
movie_data = json.load(json_open)
count=1
test_count=1
for movie_id in movie_data.keys():
    movie_vector={}
    temp_data=[]
    temp_np=np.ndarray([])
    query={"movie_id":movie_id}
    cursor=collection.find(query,{"_id":0,"review_id":1,"review_num":1,"vector":1})
    for now in cursor:
        temp_data.append(pickle.loads(now['vector']).to('cpu').detach().numpy().copy())
    if len(temp_data)>=10:
        temp_np=temp_data
        mean_movie=np.mean(temp_np,axis=0)
        movie_vector['id'] = count
        movie_vector['movie_id']=movie_id
        movie_vector['movie_title']=movie_data[movie_id]
        movie_vector['vector']=Binary(pickle.dumps(mean_movie, protocol=2))
        collection2.insert_one(movie_vector)
        count+=1
    test_count+=1

- まとめのベクトル化

In [ ]:
import GPUtil
import json
import re
import pymongo
import torch
import pickle
from pymongo import MongoClient
from bson.binary import Binary

client = MongoClient('127.0.0.1',27017)
db=client["review_data"] #database name
collection=db["matome_vector"] #collection name
#関数を使ってテキストの前処理を行う
json_open = open('matome_selected.json', 'r')
matome_selected = json.load(json_open)
start_count=0
for k,v in matome_selected.items():
    matome_vectors={}
    target_text = re.sub('[。?？!！☆★♡♥「」（）()♪・＆&/／]','',v)
    target_text = re.sub('([mMｍＭ][yYｙＹ])|マイ|まい|ﾏｲ|私の','',target_text)
    target_text = re.sub('best|Best|BEST|ｂｅｓｔ|Ｂｅｓｔ|ＢＥＳＴ|worst|Worst|WORST|ＷＯＲＳＴ|Ｗｏｒｓｔ|ｗｏｒｓｔ|([べベ][すス][とト])|ﾍﾞｽﾄ|わーすと|ワースト|ﾜｰｽﾄ|TOP|top|Top|ＴＯＰ|ｔｏｐ|Ｔｏｐ|トップ|とっぷ','',target_text)
    target_text = re.sub('(10|１０)|(10選|１０選|十選)','',target_text)
    target_text = re.sub('(邦画|邦画編)|(洋画|洋画編)','',target_text)
    target_text = re.sub('ムービー|むーびー','',target_text)
    target_text = re.sub('DVD|ＤＶＤ','',target_text)
    target_text = re.sub('化','',target_text)
    target_text = re.sub('希望','',target_text)
    target_text = re.sub('作品','',target_text)
    target_text=target_text.replace('\u3000','').replace('好きな','').replace('映画','').replace(' ','')
    with torch.no_grad():
        if len(target_text)!=0:
                target_ids = text2bertinput(input_text=target_text,tokenizer=tokenizer,max_seq_len=512)

                            #各入力idをgpuへ
                token_ids = target_ids['token_ids'].to(device)
                mask_ids = target_ids['mask_ids'].to(device)
                segment_ids = target_ids['segment_ids'].to(device)

                            #modelに各idを入力する
                bert_ouputs_average = model(
                    input_ids = token_ids,
                    attention_mask = mask_ids,
                    token_type_ids = segment_ids,
                    output_vec = 'avg_pooling')
                bert_ouput_vecs_non_grad = bert_ouputs_average
                matome_vectors['id'] = start_count+1
                matome_vectors['matome_id'] = k
                matome_vectors['matome_body'] = target_text
                matome_vectors['vector'] = Binary(pickle.dumps(bert_ouput_vecs_non_grad[0], protocol=2))
                collection.insert_one(matome_vectors)
                del token_ids
                del mask_ids
                del segment_ids
                start_count+=1
                torch.cuda.empty_cache()

- テストデータ作成

In [ ]:
#read from Mongo
import torch
import json
from pymongo import MongoClient
import numpy as np
import pickle
from bson.binary import Binary

json_open = open('matome_dic.json', 'r')
matome_to_movie = json.load(json_open)
client=MongoClient('127.0.0.1',27017) #ip, port
db=client["review_data"] #database name
combine_collection=db["combine_vector"] #collection name
matome_collection=db["matome_vector"] #collection name
review_collection=db["review_vector"] #collection name
#query={} #if you want to use only some data in database, use query to find them
cursor=matome_collection.find({},{"_id":0,"matome_id":1,"vector":1})

matome_vectors=[]
for cur in cursor:
    dic=cur
    matome_vectors.append(dic)

temp_dic={}
used_movie_id=[]
count=1
for i,now_dic in enumerate(matome_vectors):
    review_vectors=[]
    if now_dic['matome_id'] in matome_to_movie.keys():
        used_movie_id.extend(matome_to_movie[now_dic['matome_id']])
    query={"movie_id":now_dic['matome_id']} #if you want to use only some data in database, use query to find them
    review_cursor=review_collection.find(query,{"_id":0,"review_id":1,"review_num":1,"vector":1})
    for cur in review_cursor:
        dic=cur
        review_vectors.append(dic)
    for j,temp_dic in enumerate(review_vectors):
        combine_vectors={}
        c_vec = torch.cat((pickle.loads(now_dic['vector']),pickle.loads(temp_dic['vector'])),0)
        combine_vectors['id'] = count
        combine_vectors['vector'] = Binary( pickle.dumps(c_vec, protocol=2))
        combine_vectors['label'] = 1
        combine_collection.insert_one(combine_vectors)
        count+=1

In [ ]:
import random
import torch
import json
from pymongo import MongoClient
import numpy as np
import pickle
from bson.binary import Binary

json_open = open('movie_dic.json', 'r')
movie_data = json.load(json_open)
client=MongoClient('133.2.210.24',27017) #ip, port
db=client["review_data"] #database name
matome_collection=db["matome_vector"] #collection name
combine_collection=db["combine_vector"] #collection name
review_collection=db["review_vector"] #collection name
#query={} #if you want to use only some data in database, use query to find them
cursor=collection.find({},{"_id":0,"matome_id":1,"vector":1})

for cur in cursor:
    matome_data=cur
    i=0
    while i<5:
        review_vectors=[]
        rand_movie_id=random.choice(list(movie_data.keys()))
        while rand_movie_id in used_movie_id:
            rand_movie_id=random.choice(list(movie_data.keys()))
        query={"movie_id":rand_movie_id} #if you want to use only some data in database, use query to find them
        review_cursor=review_collection.find(query,{"_id":0,"review_id":1,"review_num":1,"vector":1})
        for cur in review_cursor:
            dic=cur
            review_vectors.append(dic)
        for j,temp_dic in enumerate(review_vectors):
            combine_vectors={}
            c_vec = torch.cat((pickle.loads(matome_data['vector']),pickle.loads(temp_dic['vector'])),0)
            combine_vectors['id'] = count
            combine_vectors['vector'] = Binary( pickle.dumps(c_vec, protocol=2))
            combine_vectors['label'] = 0        
            combine_collection.insert_one(combine_vectors)
            count+=1
            i+=1

- メタデータのベクトル化

In [ ]:
import torch
import json
from pymongo import MongoClient
import numpy as np
import pickle
from bson.binary import Binary

client=MongoClient('133.2.210.24',27017) #ip, port
db=client["meta_data"] #database name
collection=db["story"] #collection name
vector_collection=db["story_vector"] #collection name
#query={} #if you want to use only some data in database, use query to find them
cursor=collection.find({},{"_id":0,"タイトル":1,"解説":1,"あらすじ":1})

story=[]
for cur in cursor:
    dic=cur
    story.append(dic)

for now in story:
    count=0
    story_vector=[]
    story_dic={}
    mean_commit=0
    mean_story=0
    if '解説' in now.keys():
        section_count=1
        target_text = re.sub('[「」（）()『』]','',now['解説'])
        target_text=target_text.replace('\\n','').replace(' ','')
        target_seq=re.split('[。?？!！☆♡]',target_text)
        temp_vector=[]
        for now_seq in target_seq:
            if len(now_seq)!=0:
                with torch.no_grad():
                    with open('log.txt', 'w') as fi:

                        target_ids = text2bertinput(input_text=now_seq,tokenizer=tokenizer,max_seq_len=512)

                        #各入力idをgpuへ
                        token_ids = target_ids['token_ids'].to(device)
                        mask_ids = target_ids['mask_ids'].to(device)
                        segment_ids = target_ids['segment_ids'].to(device)

                        #modelに各idを入力する
                        try:
                            bert_ouputs_average = model(
                                    input_ids = token_ids,
                                    attention_mask = mask_ids,
                                    token_type_ids = segment_ids,
                                    output_vec = 'avg_pooling')
                            bert_ouput_vecs_non_grad = bert_ouputs_average.detach()
                            temp_vector.append([bert_ouput_vecs_non_grad])
                            del token_ids
                            del mask_ids
                            del segment_ids
                            torch.cuda.empty_cache()
                        except Exception as e:
                            print(e)
                            print(type(e))
        temp_np=np.array(temp_vector) 
        mean_commit=np.mean(temp_np,axis=0)
    if 'あらすじ' in now.keys():
        target_text = re.sub('[「」（）()『』]','',now['あらすじ'])
        target_text=target_text.replace('\\n','').replace(' ','')
        target_seq=re.split('[。?？!！☆♡]',target_text)
        temp_vector=[]
        for now_seq in target_seq:
            if len(now_seq)!=0:
                with torch.no_grad():
                    with open('log.txt', 'w') as fi:

                        target_ids = text2bertinput(input_text=now_seq,tokenizer=tokenizer,max_seq_len=512)

                        #各入力idをgpuへ
                        token_ids = target_ids['token_ids'].to(device)
                        mask_ids = target_ids['mask_ids'].to(device)
                        segment_ids = target_ids['segment_ids'].to(device)

                        #modelに各idを入力する
                        try:
                            bert_ouputs_average = model(
                                    input_ids = token_ids,
                                    attention_mask = mask_ids,
                                    token_type_ids = segment_ids,
                                    output_vec = 'avg_pooling')
                            bert_ouput_vecs_non_grad = bert_ouputs_average.detach()
                            temp_vector.append([bert_ouput_vecs_non_grad])
                            del token_ids
                            del mask_ids
                            del segment_ids
                            torch.cuda.empty_cache()
                        except Exception as e:
                            print(e)
                            print(type(e))
        temp_np=np.array(temp_vector) 
        mean_story=np.mean(temp_np,axis=0)
    if mean_commit != 0:
        story_vector.append(mean_commit)
    if mean_story != 0:
        story_vector.append(mean_story)
    story_vector=np.mean(story_vector,axis=0)
    story_dic['title'] = now['タイトル']
    story_dic['vector'] = Binary( pickle.dumps(story_vector, protocol=2))
    vector_collection.insert_one(story_dic)